In [ ]:
#Run this Every 45 days after copying the 'page_access_token' from the below script and then paste it in the short_lived_token.
#After running this script, you will get a another token in the output. Copy and paste it in the below script in the 'page_access_token' variable
import requests

app_id = '#######################'  # Replace with your app ID
app_secret = '###########################'  # Replace with your app secret
short_lived_token = '###########################################################'  # Replace with the short-lived user access token

# Exchange short-lived token for long-lived user token
url = f'https://graph.facebook.com/v17.0/oauth/access_token'
params = {
    'grant_type': 'fb_exchange_token',
    'client_id': app_id,
    'client_secret': app_secret,
    'fb_exchange_token': short_lived_token
}

response = requests.get(url, params=params)
data = response.json()

if response.status_code == 200:
    long_lived_token = data['access_token']
    print(f'Long-Lived User Token: {long_lived_token}')
else:
    print(f'Error: {data}')


Error: {'error': {'message': 'Error validating access token: Session has expired on Monday, 17-Feb-25 05:00:00 PST. The current time is Thursday, 20-Feb-25 02:54:45 PST.', 'type': 'OAuthException', 'code': 190, 'error_subcode': 463, 'fbtrace_id': 'A4AOECpzwx7lDJlgSnodPwx'}}


In [ ]:
import requests
from datetime import datetime, timedelta
import time
# import schedule
import win32com.client as win32

def fetch_form_ids(page_access_token):
    """
    Fetch all lead form IDs.
    """
    url = "https://graph.facebook.com/v17.0/me/leadgen_forms"
    params = {
        "access_token": page_access_token
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        forms = response.json().get("data", [])
        form_ids = [form['id'] for form in forms]
        return form_ids
    else:
        print("Error fetching form IDs:", response.json())
        return []

def fetch_leads_last_5_minutes(form_id, page_access_token):
    """
    Fetch leads created in the last 5 minutes for a specific form.
    """
    five_minutes_ago = datetime.utcnow() - timedelta(minutes=5)
    five_minutes_ago_str = five_minutes_ago.strftime("%Y-%m-%dT%H:%M:%S+0000")
    
    url = f"https://graph.facebook.com/v17.0/{form_id}/leads"
    params = {
        "access_token": page_access_token,
        "since": five_minutes_ago_str
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        leads = response.json().get("data", [])
        filtered_leads = [
            lead for lead in leads 
            if datetime.strptime(lead['created_time'], "%Y-%m-%dT%H:%M:%S+0000") > five_minutes_ago
        ]
        if filtered_leads:
            print(f"Fetched {len(filtered_leads)} new leads from form {form_id} in the last 5 minutes.")
            return filtered_leads
        else:
            print(f"No new leads found for form {form_id}.")
            return None
    else:
        print(f"Error fetching leads for form {form_id}:", response.json())
        return None

def parse_lead_data(lead):
    """
    Parse the lead data from the API response.
    """
    field_data = {field['name']: field['values'][0] for field in lead.get('field_data', [])}
    lead_details = {
        'name': field_data.get('full_name', 'NA'),
        'phone_number': field_data.get('phone_number', 'NA'),
        'email': field_data.get('email', 'NA'),
        'selected': field_data.get('how_much_are_you_looking_to_invest?', 'NA'),
        'company': field_data.get('company_name', 'NA'),
        'created_time': lead.get('created_time', 'NA')
    }
    return lead_details

def send_email(lead):
    """
    Send an email with all leads in a single table.
    """
    body = f'''
    META - INVESTMENT LEAD,<br><br>
    Name: {lead.get('name', 'NA')}<br>
    Contact Number: {lead.get('phone_number', 'NA')}<br>
    Email: {lead.get('email', 'NA')}<br>
    Selected: {lead.get('selected','NA')}<br>
    Message: Meta Lead <br>
    '''
    
    
    outlook = win32.Dispatch('Outlook.Application')
    mail = outlook.CreateItem(0)
    mail.To = 'saurabh.s@noveloffice.com'
    # mail.CC = 'marketing@noveloffice.in;data@noveloffice.com'
    mail.Subject = "Investment Lead"
    mail.HTMLBody = body
#     mail.SentOnBehalfOfName = 'your.email@domain.com' 
    mail.Send()

def process_leads(page_access_token):
    """
    Fetch and process leads from specific forms and send individual emails for each lead.
    """
    form_ids = fetch_form_ids(page_access_token)
    
    # Specify the two forms to fetch leads from
    selected_form_ids = ['643483218153491']
    
    for form_id in selected_form_ids:
        leads = fetch_leads_last_5_minutes(form_id, page_access_token)
        if leads:
            for lead in leads:
                # Parse lead data
                parsed_lead = parse_lead_data(lead)
                # Send email for each lead
                send_email(parsed_lead)  # Updated to send one email at a time

def job():
    """
    Scheduled job to fetch leads and send emails.
    """
    page_access_token = '##########################################################'  # Replace with your page access token
    process_leads(page_access_token)

# Schedule the job every 5 minutes
# schedule.every(10).minutes.do(job)

# Run the scheduler
print("Scheduler started. Waiting for tasks...")
while True:
#     schedule.run_pending()
    job()
    time.sleep(300)


Scheduler started. Waiting for tasks...


C:\Users\SaurabhSisodiya\AppData\Local\Temp\ipykernel_20864\2685338810.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  five_minutes_ago = datetime.utcnow() - timedelta(minutes=5)


No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
Fetched 1 new leads from form 643483218153491 in the last 5 minutes.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
No new leads found for form 643483218153491.
